In [1]:
import sqlite3
import pandas as pd

### REVIEW

In [2]:
con_rev = sqlite3.connect('../database/review.db')
df_rev = pd.read_sql_query("SELECT user_id from review", con_rev)
df_rev.head()

,user_id
0,4683161
1,24294
2,32547
3,48500
4,53601


In [3]:
df_rev.user_id.value_counts()

2867095     1536
5145589     1474
5162166     1359
2430463     1330
3724287     1252
            ... 
28262894       1
18022825       1
24920386       1
29188429       1
28673964       1
Name: user_id, Length: 2227611, dtype: int64

In [5]:
# 2867095

con_rev = sqlite3.connect('../database/user.db')
df_rev = pd.read_sql_query("SELECT * from user", con_rev)
df_rev.head()

,id,seo_name,is_featured,followers_count,followings_count,ratings_count,reviews_count,purchase_order_count,language
0,0,,0,NaN,NaN,NaN,NaN,NaN,
1,12,miko-deblascof,0,6.0,8.0,33.0,17.0,0.0,en
2,48,heiniz,0,5685.0,1748.0,1976.0,43.0,129.0,en
3,58,nicolajr,0,7.0,0.0,48.0,1.0,0.0,en
4,66,booorak,0,2.0,5.0,122.0,35.0,68.0,en


In [7]:
df_rev[df_rev.id == 2867095]

,id,seo_name,is_featured,followers_count,followings_count,ratings_count,reviews_count,purchase_order_count,language
193824,2867095,adrian_nuf,1,23424.0,5001.0,13714.0,13712.0,21.0,de


In [5]:
print('Durchscnitt: ' + str(df_rev.note_length.mean()))
print('Modus: ' + str(df_rev.note_length.mode()[0]))
print('Max: ' + str(df_rev.note_length.max()))
print('Min: ' + str(df_rev.note_length.min()))

Durchscnitt: 13.804771524073619
Modus: 1
Max: 134
Min: 1


### REVIEW_EN

In [2]:
con_rev_en = sqlite3.connect('../database/review_en.db')
df_rev_en = pd.read_sql_query("SELECT * from review_en", con_rev_en)
df_rev_en.head()

,index,id,wine_id,rating,note,created_at,likes_count,comments_count,scan_image_path,user_id,note_length,len_code
0,0,3975,1101258,4.0,Excellent full bodied wine,2011-07-19T21:21:22.000Z,2.0,1.0,images.vivino.com/labels/1309631550_2382.jpg,32547,4,en
1,1,4202,76378,5.0,The Real price at phillipson is around 1299 dk...,2011-08-06T20:52:53.000Z,0.0,1.0,images.vivino.com/labels/1311959128_1819.jpg,48500,39,en
2,2,4261,1231210,5.0,"Clean, crisp and full of nuance and elegance, ...",2011-08-11T08:06:39.000Z,3.0,1.0,images.vivino.com/labels/1312622458_6592.jpg,53601,17,en
3,3,4264,1178663,4.0,"Crisp and clean Champagne, a certain fave amon...",2011-08-11T09:28:07.000Z,25.0,2.0,images.vivino.com/labels/1312966707_6129.jpg,53601,22,en
4,4,4354,17998,4.0,It is not a champagne type wine! It is a red w...,2011-08-12T16:53:19.000Z,0.0,1.0,images.vivino.com/labels/1311842730_8268.jpg,47873,12,en


In [8]:
print('Durchscnitt: ' + str(df_rev_en.note_length.mean()))
print('Modus: ' + str(df_rev_en.note_length.mode()[0]))
print('Max: ' + str(df_rev_en.note_length.max()))
print('Min: ' + str(df_rev_en.note_length.min()))

Durchscnitt: 15.015858303487168
Modus: 1
Max: 121
Min: 1


In [4]:
df_rev_en[df_rev_en.note.str.contains('vivino', case=False)]

,index,id,wine_id,rating,note,created_at,likes_count,comments_count,scan_image_path,user_id,note_length,len_code
16,16,6495,75318,5.0,"really a 1993, but Vivino didnt give me the ch...",2011-12-19T21:51:02.000Z,1.0,1.0,images.vivino.com/labels/1324324258_7349.jpg,110863,12,en
347,347,184222,20647,4.0,Vintage 1992. But Vivino does not handle real ...,2012-07-19T12:30:47.000Z,0.0,1.0,images.vivino.com/labels/1319832551_2356.jpg,8968,11,en
1037,1039,514549,1654,4.0,This is Not s champagne Vivino! Get a grip!,2012-10-13T07:16:17.000Z,2.0,1.0,images.vivino.com/labels/1350112565_2893.jpg,388804,9,en
1044,1046,518144,85758,3.0,"Vivino, The ranking criteria was Bad. Now got ...",2012-10-13T18:38:15.000Z,0.0,0.0,images.vivino.com/labels/1350153482_9011.jpg,423501,11,en
4729,4747,1673144,1099392,5.0,Looking Forward To Taste This Port. Anyone @ V...,2013-02-23T13:51:44.000Z,10.0,5.0,images.vivino.com/labels/1361627481_9583.jpg,16274,18,en
...,...,...,...,...,...,...,...,...,...,...,...,...
3975675,4114983,257355928,97275,4.5,Outstanding wine at its peak!\n\n(again Vivino...,2022-10-14T11:14:59.000Z,78.0,7.0,images.vivino.com/labels/artCcov3REy8EQTHwMexH...,13475212,11,en
3976083,4115392,257512158,2332011,3.7,Actually very good Chardonnay. Definitely over...,2022-10-15T17:08:57.000Z,0.0,0.0,images.vivino.com/labels/ESoEe3uyRnOd8Cts6uMvy...,47083268,22,en
3976542,4115852,257702285,657,4.0,"Wrong, Vivino! I like this one, and for the pi...",2022-10-17T00:15:46.000Z,1.0,0.0,images.vivino.com/labels/zLUK4S7ORD6I5SONTEutq...,27676569,19,en
3976589,4115899,257718048,1133931,3.5,Very very pineapple - complete with the prickl...,2022-10-17T09:44:24.000Z,3.0,0.0,images.vivino.com/labels/IAiJm_X0RieFQjIiU-e31...,2920930,52,en


In [4]:
con_rev = sqlite3.connect('../database/review_en.db')
df_rev = pd.read_sql_query("SELECT note_length FROM review_en", con_rev)
df_rev.head()

,note_length
0,4
1,39
2,17
3,22
4,12


In [5]:
df_rev.note_length.mean()

15.015858303487168

### REVIEW_EN_SENTIMENT

In [2]:
con_rev_en_sent = sqlite3.connect('../database/review_en_sentiment.db')
df_rev_en_sent = pd.read_sql_query("SELECT * FROM review_en_sentiment", con_rev_en_sent)
df_rev_en_sent.head()


KeyboardInterrupt

